# Project Tree Visualization Notebook

This notebook explores methods to visualize the directory structure of the JennAI project. It's useful for understanding the project layout, especially when working with a monorepo structure.

We'll primarily focus on using the `eza` command-line tool, which provides excellent tree views, and also consider Python-native approaches.

## 1. Environment Setup & Path Configuration

Ensure the JennAI project root is added to `sys.path` so we can potentially import project-specific utilities if needed, and correctly reference the project root.

In [ ]:
import sys
from pathlib import Path

# Determine the JennAI project root dynamically
# Assumes this notebook is in 'JennAI/admin/'
current_notebook_path = Path(__file__).resolve()
jennai_root = current_notebook_path.parent.parent # Go up two levels from admin/notebook.ipynb

if str(jennai_root) not in sys.path:
    sys.path.insert(0, str(jennai_root))

print(f"JennAI project root set to: {jennai_root}")

## 2. Visualizing with `eza --tree`

`eza` is a modern replacement for `ls` with great features, including a powerful tree view. Ensure `eza` is installed on your system (`sudo apt install eza` or `brew install eza`, etc.).

In [ ]:
import subprocess

def display_project_tree_with_eza(project_path: Path):
    """Runs 'eza --tree' for the given project path and prints the output."""
    print(f"Attempting to display project tree for: {project_path} using 'eza --tree'...")
    try:
        # Check if eza is installed by trying to get its version
        subprocess.run(["eza", "--version"], check=True, capture_output=True, text=True)
        
        # Run eza --tree
        # You can add more eza options here, e.g., --level=3, --ignore-glob=".git|__pycache__"
        result = subprocess.run(
            ["eza", "--tree"], 
            cwd=project_path, 
            check=True, 
            capture_output=True, 
            text=True
        )
        print("\n------------------- Project Tree (eza) -------------------")
        print(result.stdout.strip())
        print("--------------------------------------------------------")
        print("Successfully displayed project tree using 'eza --tree'.")
    except FileNotFoundError:
        print("'eza' command not found. Please install eza to use this feature.")
    except subprocess.CalledProcessError as e:
        print(f"'eza --tree' command failed with error: {e}")
        print(f"  Stderr: {e.stderr}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Display the tree for the current JennAI project root
display_project_tree_with_eza(jennai_root)

## 3. Python-native Tree Visualization (Placeholder)

If `eza` is not available or for a more programmatic approach, we can use Python's `pathlib` and `os` modules to build a tree view. This is more involved but offers more control.

In [ ]:
from pathlib import Path
import os # For os.sep, though Path handles separators well

def display_tree_python_native(start_path: Path, level: int = 0, indent_char: str = "│   ", last_indent_char: str = "    ", prefix: str = "", ignore_dirs: list = None, ignore_files: list = None):
    """Recursively prints a directory tree structure using Python's pathlib."""
    if ignore_dirs is None:
        ignore_dirs = ['.git', '__pycache__', '.pytest_cache', '.venv', 'venv', 'env', 'ENV', 'logs', 'allure-results', 'allure-report', '.ipynb_checkpoints']
    if ignore_files is None:
        ignore_files = [] # e.g., ['.DS_Store']

    if level == 0:
        print(f"{start_path.name}/")

    # Get directory contents, filter out ignored items, and sort
    try:
        contents = sorted([item for item in start_path.iterdir() 
                           if (item.is_dir() and item.name not in ignore_dirs) or 
                              (item.is_file() and item.name not in ignore_files)])
    except PermissionError:
        print(f"{prefix}└── [Permission Denied]")
        return
    except FileNotFoundError:
        print(f"{prefix}└── [Path Not Found]")
        return

    pointers = ["├── "] * (len(contents) - 1) + ["└── "]

    for pointer, path_item in zip(pointers, contents):
        print(f"{prefix}{pointer}{path_item.name}{'/' if path_item.is_dir() else ''}")

        if path_item.is_dir():
            extension = indent_char if pointer == "├── " else last_indent_char
            display_tree_python_native(path_item, level + 1, prefix=prefix + extension, ignore_dirs=ignore_dirs, ignore_files=ignore_files)

# Display the tree for the current JennAI project root using the Python-native function
print("\n------------------- Project Tree (Python Native) -------------------")
display_tree_python_native(jennai_root, ignore_dirs=['.git', '__pycache__', '.pytest_cache', '.venv', 'venv', 'env', 'ENV', 'logs', 'allure-results', 'allure-report', '.ipynb_checkpoints', 'anaconda_projects', '.virtual_documents', '.trash'])
print("--------------------------------------------------------------------")